In [ ]:
# !pip install PyPDF2
# !pip install python-docx
!pip install groq
# !pip install sentence-transformers
# !pip install fitz
# !pip uninstall fitz
# !pip install pymupdf
# !pip install python-docx
# !pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import docx
import os
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # A cost-effective embedding model

# Function to extract text from DOCX
def extract_text_from_docx(filepath):
    doc = docx.Document(filepath)
    return "\n".join([para.text for para in doc.paragraphs])

# Function to extract text from PDF
def extract_text_from_pdf(filepath):
    text = ""
    with fitz.open(filepath) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf.load_page(page_num)
            text += page.get_text()
    return text

# Function to process files and create embeddings
def process_documents(folder_path):
    embeddings = []
    metadata = []

    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)

        if filename.endswith('.docx'):
            text = extract_text_from_docx(filepath)
        elif filename.endswith('.pdf'):
            text = extract_text_from_pdf(filepath)
        else:
            continue

        # Split text into paragraphs and generate embeddings
        paragraphs = text.split("\n\n")
        for para in paragraphs:
            if para.strip():
                embedding = model.encode(para)
                embeddings.append(embedding)
                metadata.append({'filename': filename, 'paragraph': para})

    return embeddings, metadata

# Example usage
folder_path = "/content/"
embeddings, metadata = process_documents(folder_path)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import faiss
import numpy as np

# Convert embeddings list to a NumPy array
embedding_matrix = np.array(embeddings)

# Create FAISS index
dimension = embedding_matrix.shape[1]  # Embedding dimension (e.g., 384 for MiniLM)
index = faiss.IndexFlatL2(dimension)  # L2 distance index
index.add(embedding_matrix)  # Add embeddings to index

# Save index and metadata for later usage
faiss.write_index(index, "index.faiss")

import pickle
with open("metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)


In [ ]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load FAISS index and metadata
index = faiss.read_index("index.faiss")
with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Load embedding model again
model = SentenceTransformer('all-MiniLM-L6-v2')

def retrieve_relevant_context(query, top_k=5):
    # Generate embedding for the query
    query_embedding = model.encode(query).reshape(1, -1)

    # Perform similarity search
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the top-k relevant paragraphs
    relevant_context = [metadata[idx]['paragraph'] for idx in indices[0]]

    return relevant_context

# Example usage
query = "What was the purpose of the Rosson Mine deed from 1882?"
relevant_context = retrieve_relevant_context(query)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import requests
from groq import Groq
def query_groq_llm(prompt):

    # Initialize Groq client
    client = Groq(api_key="Enter Your Key")

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama3-8b-8192"
    )
    return chat_completion.choices[0].message.content

# Example: Construct prompt with retrieved context and query LLM
context_text = "\n\n".join(relevant_context)
llm_prompt = f"Given the context below, answer the following question:\n\n{query}\n\nContext:\n{context_text}"

# Query Groq LLM
response = query_groq_llm(llm_prompt)
print(response)



The Rosson Mine deed from 1882 was a warranty deed from James Addison Reavis to R.L. Rosson, transferring ownership of approximately 640 acres of land in Maricopa County, Arizona. Specifically, the deed granted R.L. Rosson the North half of Section 35, the South East quarter of Section 35, and the South East quarter of Section 36, all situated in Township 3 North, Range 2 East of the G&SR Base and Meridian according to U.S. Survey.


In [ ]:
def main():
    while True:
        query = input("Enter A query")
        if query.lower() == 'exit':
            break

        # Retrieve context
        relevant_context = retrieve_relevant_context(query)

        # Construct prompt and query LLM
        context_text = "\n\n".join(relevant_context)
        llm_prompt = f"Given the context below, answer the following question:\n\n{query}\n\nContext:\n{context_text}"
        response = query_groq_llm(llm_prompt)

        # Print the LLM's response
        print("\nResponse:\n")
        print(response)
        print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    main()


Enter A queryI want a template deed that takes reference from existing deeds for transferring assets to someone else

Response:

Here is a template deed for transferring assets to someone else, based on the existing deeds:

**Template Deed**

This Indenture, made this [insert date] day of [insert month], in the year of our Lord [insert year], between [insert grantor's name], [insert grantor's address], the County of [insert county], and the State of [insert state], party of the first part, and [insert grantee's name], [insert grantee's address], party of the second part, Witnesseth:

That the said party of the first part, for and in consideration of the sum of [insert consideration], lawful money of the United States of America, paid to the said party of the first part in hand by the said party of the second part, the receipt whereof is hereby acknowledged, does by these presents grant, bargain, sell and convey unto the said party of the second part, and to their heirs and assigns fore

KeyboardInterrupt: Interrupted by user